# Developing an analytical platform for evaluating the role of forest biorefineries in achieving a sustainable bioeconomy

Objectives:
--------------
This script aims to assess the environmental performance of various biochemicals by using Brightway2 frameworks. Specifically, it evaluates the environmental footprints of key bio-based chemicals.

The targeted biochemicals include:<br>
Bioethanol<br>
Furfural<br>
Vanillin<br>
These chemicals are analyzed in terms of their production processes, greenhouse gas emissions, and energy consumption.


Key Indicators:
--------------
The code focuses on major environmental performance indicator:

Greenhouse Gas Emissions: Evaluated using the Life Cycle Assessment (LCA) framework to capture the global warming potential (GWP) of each chemical process.


Requirements:
-------------
The following Python libraries and tools are essential for running the LCA script:

1. `brightway2`: A framework for Life Cycle Assessment (LCA) in Python. Install with: `pip install brightway2==2.3`

2. `bw2analyzer`: Provides advanced analysis capabilities for Brightway2. Install with: `pip install bw2analyzer==0.10`

3. `bw2calc`: Used for performing calculations in Brightway2. Install with: `pip install bw2calc==1.8.0`

4. `bw2data`: Handles the data storage for Brightway2. Install with: `pip install bw2data==3.6.2`

5. `stats-arrays`: A package for uncertainty analysis using Brightway2. Install with: `pip install stats-arrays==0.6.5`

6. `pandas`: Used for data manipulation and analysis, particularly for reading and writing data from various file formats. Install with: `pip install pandas==1.3.3`

7. `numpy`: Used for numerical operations and handling of arrays. Install with: `pip install numpy==1.21.2`

8. `plotly`: A graphing library used to create interactive visualizations. Install with: `pip install plotly==5.15.0`

## Life cycle assessment

In [ ]:
from brightway2 import* #import packages; 
from bw2analyzer import ContributionAnalysis
import stats_arrays
from bw2analyzer import traverse_tagged_databases
from bw2analyzer.tagged import recurse_tagged_database
import collections
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.offline import plot

In [ ]:
class biorefinery_LCA:
    def __init__ (self,project_name):
        self.project_name=project_name
        projects.set_current(self.project_name)
        print (projects.current)  
        self.analysis_done = False  # Initialize to False when object is created
        self.LCA_results_dict = {}  # Ensure this dictionary is properly initialized
        
        ##set up default dataset (biosphere3) and LCIA methods for current project
        bw2setup()
        
    def import_db (self,db_path_name_dict):
        self.db_path_name_dict=db_path_name_dict
        
        for key, val in self.db_path_name_dict.items():
            self.db_location=val
            self.db_name=key
            
            import_obj=SingleOutputEcospold2Importer(
                self.db_location,
                self.db_name)
            import_obj.apply_strategies()
            import_obj.statistics()
            
            ##write database
            import_obj.write_database()
            
    def import_foreground (self,forground_db_name,input_file_path,db_mapping_dict):
        ##prepare foreground db
        self.forground_db_name=forground_db_name #this must be identical to the "Database" name in the input spreadsheet 
        self.input_file_path=input_file_path
        self.db_mapping_dict=db_mapping_dict #{db_name:('field_1','field_2',...)}
        
        self.import_foreground_obj=ExcelImporter(self.input_file_path)
        self.import_foreground_obj.apply_strategies()
        for db_name,fields_to_map in self.db_mapping_dict.items():
            if db_name=='self':
                self.import_foreground_obj.match_database(fields=fields_to_map) #link within the foreground processes
            else:
                self.import_foreground_obj.match_database(db_name,fields=fields_to_map) #mapping to processes in other db
        self.import_foreground_obj.statistics()
        self.import_foreground_obj.write_excel(only_unlinked=True)
        
        self.import_foreground_obj.write_database()
        self.foreground_db=Database(forground_db_name)
        
# Vanillin can be modified to bioethanol, furfural, or any other biochemical under study 
    def calc_lca(self, lcia_methods, db, FU_codes, vanillin_production, calc_done=False):
            self.FU_codes = FU_codes
            self.lcia_methods = lcia_methods
            self.calc_done = calc_done

            ##create dict to store: (1) LCA results, (2) top processes (including backgr db)
            self.LCA_results_dict = {}
            self.top_processes_dict = {}

            ##create a ContributionAnalysis object
            self.contribut_anal_obj = ContributionAnalysis()

            # Iterate over each functional unit code
            for FU_name, FU_code in self.FU_codes.items():
                # Find the activity in the database corresponding to the FU code
                FU_activity = [act for act in db if act['code'] == FU_code][0]
                #amount_FU = 1  # or any other amount that you wish to use

                # Set the amount of FU based on vanillin production 
                amount_FU = vanillin_production[FU_name]  # Use the actual vanillin production value

                                
                # Iterate over each LCIA method
                for method in self.lcia_methods:
                    self.lca = LCA({FU_activity: amount_FU}, method)
                    self.lca.lci()
                    self.lca.lcia()

                    # Store the results under the FU name
                    if FU_name not in self.LCA_results_dict:
                        self.LCA_results_dict[FU_name] = {}
                    self.LCA_results_dict[FU_name][method] = self.lca.score

                    if FU_name not in self.top_processes_dict:
                        self.top_processes_dict[FU_name] = {}
                    self.top_processes_dict[FU_name][method] = self.contribut_anal_obj.annotated_top_processes(self.lca)

            ##update the label to True
            self.calc_done = True

    def print_lca_results(self):
            # Print out the LCA results for all products
        for product, methods_results in self.LCA_results_dict.items():
            print(f"LCA Results for {product}:")
            for method, score in methods_results.items():
                print(f"  Method: {method}, Score: {score}")
            print()  # Adds a blank line for better readability

    
    def analyze_lca (self,impact_of_interest,n_top_items=5,analysis_done=False):
        self.impact_of_interest=impact_of_interest
        self.n_top_items=n_top_items #number of top items (e.g., top processes) of interest
        assert self.impact_of_interest in self.LCA_results_dict.keys(), "This method is not in your LCIA method list!"
        self.analysis_done= True #analysis_done
        ##create a dict to store impact results by 'group_tag' (technoshpere exchanges only)
        self.techno_impact_results_grouped=collections.defaultdict(list)
        
        
        while not self.analysis_done: #if analysis has not been done yet
            ##find top technosphere processes (including background db)
            self.top_processes={self.impact_of_interest : self.top_processes_dict[self.impact_of_interest][:self.n_top_items+1]}
            ##group the results by tag
            for exc in self.FU_activity.technosphere():
                self.lca2=LCA({exc.input : exc['amount']},
                               self.impact_of_interest)
                self.lca2.lci()
                self.lca2.lcia()
                self.techno_impact_results_grouped[exc['group_tag']].append(self.lca2.score)
            
            self.techno_impact_results_grouped={key : sum(val) for key, val in self.techno_impact_results_grouped.items()}
            
            ##finally, update the label to True
            self.analysis_done=True
            
            
    def gen_fig (self,data_to_plot_dict,plot_type,x_label,y_label):
        assert self.analysis_done==True,"please run the 'analyze_lca' method first!"        
        
        self.data_to_plot_dict=data_to_plot_dict #input data must be in the dict form
        self.df_for_plot=pd.DataFrame.from_dict([self.data_to_plot_dict]) #convert to a pandas dataframe for plotting
        
        if plot_type=='bar plot':
            ##bar chart for top processes (x-axis=impact results, y-axis=process names)
            ax=self.df_for_plot.plot(kind='barh')
            ax.set_xlabel(x_label, labelpad=20, weight='bold', size=12)
            ax.set_ylabel(y_label, labelpad=20, weight='bold', size=12)
        elif plot_type=='waterfall chart':
            ##waterfall chart for "group_tag" results (x-axis=group names, y-axis=impact results)
            #prepare param for Plotly
            measure_type=['relative']*len(self.df_for_plot.columns) #first n columns of the waterfall chart 
                #should represent relative changes from each column (e.g., waste treatment) of the dataframe
            measure_type.append('total') #add a represenation of net impact which is the total (sum) of individial changes
            self.df_for_plot['net']=self.df_for_plot.values.sum()#add a column corresponding to net impact
            values_to_plot=[val for lst in self.df_for_plot.values.tolist() for val in lst] #flatten the nested list of df.values
            values_as_text=[str(round(val,1)) for val in values_to_plot]
            waterfall_x_label=list(self.df_for_plot.columns)

            
            #create the plot object
            fig = go.Figure(go.Waterfall(
                    name = '-'.join(self.impact_of_interest), orientation = "v",
                    measure = measure_type,
                    x = waterfall_x_label,
                    textposition = "outside",
                    text = values_as_text,
                    y = values_to_plot,
                    connector = {"line":{"color":"rgb(63, 63, 63)"}},
                    ))
            
            fig.update_layout(
                    showlegend = True)
            
            fig.show() #only works if you are using Jupyter Notebook
            #plot(fig) #works for spyder, will create a temp html file to host the fig
            
        else:
            print ("Please choose 'bar plot' or 'waterfall chart' ")
            
            
    def gen_report (self):
        assert self.analysis_done==True,"please run the 'analyze_lca' method first!"
        pass
    
    

In [ ]:
lca_obj=biorefinery_LCA('LCA_module')

In [ ]:
import brightway2 as bw
ei37dir ="./datasets e.g. ecoinvent"    #Set the directory path to the local 'datasets' folder containing ecoinvent files
if 'ecoinvent 3.7 cutoff' in bw.databases:
    print("Database has already been imported")
else:
    ei37 = bw.SingleOutputEcospold2Importer(ei37dir, 'ecoinvent 3.7 cutoff') # You can give it another name of course
    ei37.apply_strategies()
    ei37.statistics()
    print("ei37 has been defined")
    ei37.write_database() # This will take some time.    
    

In [ ]:
# Load the biosphere database
biosphere = bw.Database('biosphere3')

# Check the number of flows in the biosphere database
print("Number of flows in biosphere3:", len(biosphere))

In [ ]:
 bw.projects.output_dir

In [ ]:
##load foreground LCI #OOO#
forground_db_name="Vanillin"
input_file_path= "./input/Vanillin.xlsx" # Set the path to the Vanillin.xlsx input file (assumed to be in the local 'input' folder)

db_mapping_dict={"self":('name', 'unit', 'location','reference product'),
                 "ecoinvent 3.7 cutoff":('name', 'unit', 'location','reference product')}
lca_obj.import_foreground(forground_db_name,input_file_path,db_mapping_dict)

In [ ]:
#OOO#
db_name = 'Vanillin'

# Check if the database exists
if db_name in databases:
    print(f"Database '{db_name}' exists.")
    # List all activities in the database
    db = Database(db_name)
    for activity in db:
        print(activity)
else:
    print(f"Database '{db_name}' does not exist.")

In [ ]:
# Initialize an empty dictionary to store vanillin production values 
vanillin_production_values = {}

# Loop through the database to find all production exchanges and store them in the dictionary 
for activity in db:
    for exc in activity.exchanges():
        if exc['type'] == 'production' and 'Vanillin' in exc['name']:
            # Add the production name and amount to the dictionary
            vanillin_production_values[exc['name']] = exc['amount']

# Print the generated dictionary
print(vanillin_production_values)

In [ ]:
for activity in db:
    for exc in activity.exchanges():
        if exc['type'] == 'production':
            print(exc['name'], exc['amount'])

In [ ]:
from brightway2 import methods
print(methods)

In [ ]:
list(methods)

In [ ]:
if not lca_obj.foreground_db:
    print("The database is empty or not loaded properly.")
else:
    print(f"The database contains {len(lca_obj.foreground_db)} activities.")


In [ ]:
lca_obj.foreground_db = Database("Vanillin")  # Make sure the database is initialized correctly #OOO#

In [ ]:
# Generate functional unit codes from 100% woodchips to 1% woodchips #OOO#
FU_codes = {f'Vanillin {i}% production': f'ThisIsFU{i}' for i in range(100, 0, -1)}

In [ ]:
# Display the generated functional unit codes
for key, value in FU_codes.items():
    print(f'{key}: {value}')

In [ ]:
#lcia_methods=[('CML 2001 (obsolete)', 'climate change', 'GWP 100a')]
lcia_methods=[('TRACI (obsolete)', 'environmental impact', 'global warming')]

In [ ]:
lca_obj.calc_lca(lcia_methods, lca_obj.foreground_db, FU_codes, vanillin_production_values)##calculate LCA results #OOO#

In [ ]:
# Print out the LCA results for all products
for product, results in lca_obj.LCA_results_dict.items():
    print(f"LCA Results for {product}:")
    for method, score in results.items():
        print(f"  Method: {method}, Score: {score}")
    print()  # Adds a blank line for better readability